In [3]:
%%capture
%matplotlib inline
import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'
#import matplotlib.pyplot as plt
#plt.plot([0,1],[1,0])
from neuronunit.optimisation.optimization_management import test_all_objective_test
import numpy as np
from IPython.display import display, HTML
from neuronunit.plottools import plot_score_history1
from neuronunit.optimisation.optimization_management import check_binary_match, TSD
import pandas as pd
from collections import OrderedDict
import pickle


from deap import creator
#creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
from deap import base
import array
#creator.create("FitnessMin", base.Fitness, weights=tuple(-1.0 for i in range(0,10)))
#creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)

In [4]:
cells = pickle.load(open("processed_multicellular_constraints.p","rb"))

ncl5 = TSD(cells["Neocortex pyramidal cell layer 5-6"])
vr = ncl5["RestingPotentialTest"].observation['mean']
#vt = ncl5["InjectedCurrentAPThresholdTest"].observation['mean']
#C = ncl5["CapacitanceTest"].observation['mean']

ncl5.keys()

dict_keys(['RheobaseTest', 'InputResistanceTest', 'TimeConstantTest', 'CapacitanceTest', 'RestingPotentialTest', 'InjectedCurrentAPWidthTest', 'InjectedCurrentAPAmplitudeTest', 'InjectedCurrentAPThresholdTest'])

# Simulated Data 
and select model parameters that are free to vary

In [5]:
%%capture
#fps = ['c','a','b','d','vPeak','k','C','vt']
model_type="NEURONHH"
#sim_tests, OM, target = test_all_objective_test(fps,model_type=model_type)

# Meta Parameters

In [6]:
MU = 4 
NGEN = 150
results = {}
for t in cells['Olfactory bulb (main) mitral cell']:
    print(t.observation)
    
cells.pop('Olfactory bulb (main) mitral cell',None)

In [7]:
import joblib

model_type="HH"
fps = ['g_L','g_Na','g_K']   
for k,v in cells.items():
    results[k] = TSD(v).optimize(backend=model_type,\
            protocol={'allen': False, 'elephant': True},\
                MU=MU,NGEN=NGEN,plot=True,free_parameters=fps,hold_constant={'vr':vr})

    break
    #with open('neo.p','wb') as f:
    #    joblib.dump(results, f)    

        #pickle.dump(results,f)


KeyError: 'HH'

In [11]:
opts = []
for v in results.values():
    if v is not None:
        opt = v['hof'][0].dtc
        opts.append(opt)
        agreement = opt.get_agreement().agreement
        display(agreement)


,RheobaseTest,InputResistanceTest,TimeConstantTest,CapacitanceTest,RestingPotentialTest,InjectedCurrentAPWidthTest,InjectedCurrentAPAmplitudeTest,InjectedCurrentAPThresholdTest,total
observations,189.24 pA,107.080327644332 Mohm,24.5021946169772 ms,89.7960714285714 pF,-65.2261863636364 mV,1.31895278450363 ms,86.364525297619 mV,-47.5985714285714 mV,NaN
predictions,98.052978515625 pA,46.324486119296175 Mohm,7.292033335434033 ms,157.4120718070218 pF,-68.2481434599156 mV,0.8400000000000001 ms,62.79851764232387 mV,-29.103581197808637 mV,NaN
scores,0.286575,0.835792,0.859973,1.57237,0.622743,0.841473,2.9071,4.09343,36.087851


In [7]:
#opt.attrs['vt'] =
#opt.attrs['C'] =

#from neuronunit.optimisation.optimization_management import feature_mine


In [8]:
#OM = opt.dtc_to_opt_man()
#out = OM.random_search(opt,100)
#display(out['frame'])
opt = results['pf'][0].dtc

opt = feature_mine(opt)

KeyError: 'pf'

# Analyse Results

In [ ]:
with open('neo.p','wb') as f:
    pickle.dump(opt,f)
hof = results['hof'][0].dtc
agreement = opt.get_agreement().agreement

display(agreement)

# Compare scores above to random scores below

In [ ]:
# This should be more succint than above.
#display(out['frame'])
MU = 10 
NGEN = 50
model_type = "HH"
from neuronunit.optimisation import model_parameters
fps = model_parameters.MODEL_PARAMS["HH"].keys()

print(fps)
results = ncl5.optimize(backend=model_type,\
        protocol={'allen': False, 'elephant': True},\
            MU=MU,NGEN=NGEN,plot=True,free_parameters=fps,hold_constant={'Vr':vr})

In [ ]:
out['best_random_sum_total']

# Look at evolution History

In [ ]:
plt = plot_score_history1(results)
from neuronunit.optimisation.optimization_management import check_binary_match

In [ ]:
model = opt.dtc_to_model()
#check_binary_match(opt,target,snippets=True)
#target = OM.format_test(target)
#simulated_data_tests = target.tests

In [ ]:
#check_binary_match(opt,target,snippets=False)

In [ ]:
try:
    opt.attrs.pop('dt',None)
    opt.attrs.pop('Iext',None)
except:
    pass


In [ ]:
display(pd.DataFrame([opt.attrs]))
import copy
temp = {}
for k in opt.attrs.keys():
    temp[k] = target.attrs[k]
display(pd.DataFrame([temp]))

In [ ]:
display(pd.DataFrame([{k.name:v for k,v in opt.SA.items()}]))

What where the values of model parameters that where held constant?


In [ ]:
df0 = opt.dtc_to_model().default_attrs
df1 = target.dtc_to_model().default_attrs
hc = {}

try:
    df0.attrs.pop('dt',None)
    df0.attrs.pop('Iext',None)
    opt.attrs.pop('dt',None)
    opt.attrs.pop('Iext',None)

except:
    pass

for k,v in df0.items():
    if k not in opt.attrs.keys():
        assert df0[k] == df1[k]
        hc[k] = v        
display("Held constant:")
display(pd.DataFrame([hc]))        

If the Pareto Front encircles the best solution without sampling directly on top of it.
Does piercing the center get us closer to the hall of fame?
Below, plot HOF[0]/PF[0] are they the same model? 

In [ ]:
# check_binary_match(opt,hof,snippets=True)


# Exploring the neighbourhood of 
the Optimal solution is now syntatically easy

## Make ranges to explore:

In [ ]:
from neuronunit.optimisation.model_parameters import MODEL_PARAMS
a_range = MODEL_PARAMS["IZHI"]['a']
grid_a = np.linspace(a_range[0],a_range[1],25)

import copy


## Mutate a parameter in a dimension of interest.

In [ ]:
opt_sum0 = np.sum(opt.SA.values)
for_scatter0 = (opt.attrs['a'],opt_sum0)
from tqdm import tqdm
collect = []
mutant = copy.copy(opt)
for a in tqdm(grid_a):
    # non random mutation
    mutant.attrs['a'] = a
    # Evaluate NU test suite
    mutant.self_evaluate()
    # sum components (optional)
    fit = np.sum(mutant.SA.values)
    collect.append(fit)
plt.plot(grid_a,collect)
plt.scatter(for_scatter0[0],for_scatter0[1],label='optima')
    

The above plot seemed to have multiple steep wells of low error about the optima.

It might not be reasonable to expect to sample every such well, as the stochastic and non exhuastive sampling in the GA means it is not garunteed to sample small and and focused pockets of error change.

In [ ]:
b_range = MODEL_PARAMS["IZHI"]['b']
grid_b = np.linspace(b_range[0],b_range[1],30);

In [ ]:
opt_sum = np.sum(opt.SA.values)
for_scatter1 = (opt.attrs['b'],opt_sum)
from tqdm import tqdm
collect = []
mutant = copy.copy(opt)
for b in tqdm(grid_b):
    # non random mutation
    mutant.attrs['b'] = b
    # Evaluate NU test suite
    mutant.self_evaluate()
    # sum components (optional)
    fit = np.sum(mutant.SA.values)
    collect.append(fit)
plt.plot(grid_b,collect)
plt.scatter(for_scatter1[0],for_scatter1[1],label='optima')


# Compare the match of passive waveforms

In [ ]:

#from neuronunit.optimisation.optimization_management import inject_and_plot_passive_model
#inject_and_plot_passive_model(opt)
import quantities as pq
tm = target.dtc_to_model()

model = opt.dtc_to_model()
uc = {'amplitude':-10*pq.pA,'duration':500*pq.ms,'delay':100*pq.ms}
model.inject_square_current(uc)
vm1 = model.get_membrane_potential()
tm.inject_square_current(uc)
vm0 = tm.get_membrane_potential()
plt.plot(vm1.times, vm1.magnitude, c='b',label=str('opt ADEXP'))#+str(model.attrs['a']))
plt.plot(vm0.times, vm0.magnitude, c='r',label=str('target HH'))#+str(model.attrs['a']))

plt.show()